In [1]:
from itertools import chain
from collections import defaultdict
import copy
import math
from math import sqrt, sin, cos, pi
import random

from sklearn.neighbors import KDTree
import numpy as np

import plotly
import plotly.graph_objs as go

plotly.offline.init_notebook_mode()

# EPS = 1e-10
# EPS = 1e-8
# EPS = 1e-7/2
EPS = 1e-7
# EPS = 1e-6 / 2
# EPS = 1e-6
# EPS = 1e-5
LESS_EPS = EPS
# LESS_EPS = 1e-6
# LESS_EPS = 1e-5
# LESS_EPS = 1e-4
# LESS_EPS = 1e-2

def same(a, b, eps=EPS):
    return abs(a - b) < eps

def same_points(a, b, eps=EPS):
    return same(a[0], b[0], eps) and same(a[1], b[1], eps) and same(a[2], b[2], eps)

def normed(p):
    norm = sqrt(sum([x**2 for x in p]))
    return np.array(list(p)) / norm

# https://math.stackexchange.com/a/4112622/376363
def copysign(a, b):
    if b >= 0:
        return abs(a)
    return -abs(a)

# https://math.stackexchange.com/a/4112622/376363
def perp1(p):
    x,y,z = p
    ux,uy,uz = (copysign(z, x), copysign(z, y), -copysign(abs(x) + abs(y), z))
    return normed(np.array([ux, uy, uz]))

# https://math.stackexchange.com/a/4112622/376363
def perp2(p):
    p2 = perp1(p)
    p3 = np.cross(p, p2)
    return normed(p3)

def rand_rotation_matrix(deflection=1.0, randnums=None):
    """
    Creates a random rotation matrix.

    deflection: the magnitude of the rotation. For 0, no rotation; for 1, competely random
    rotation. Small deflection => small perturbation.
    randnums: 3 random numbers in the range [0, 1]. If `None`, they will be auto-generated.
    """
    # from http://www.realtimerendering.com/resources/GraphicsGems/gemsiii/rand_rotation.c

    if randnums is None:
        randnums = np.random.uniform(size=(3,))

    theta, phi, z = randnums

    theta = theta * 2.0*deflection*np.pi  # Rotation about the pole (Z).
    phi = phi * 2.0*np.pi  # For direction of pole deflection.
    z = z * 2.0*deflection  # For magnitude of pole deflection.

    # Compute a vector V used for distributing points over the sphere
    # via the reflection I - V Transpose(V).  This formulation of V
    # will guarantee that if x[1] and x[2] are uniformly distributed,
    # the reflected points will be uniform on the sphere.  Note that V
    # has length sqrt(2) to eliminate the 2 in the Householder matrix.

    r = np.sqrt(z)
    Vx, Vy, Vz = V = (
        np.sin(phi) * r,
        np.cos(phi) * r,
        np.sqrt(2.0 - z)
        )

    st = np.sin(theta)
    ct = np.cos(theta)

    R = np.array(((ct, st, 0), (-st, ct, 0), (0, 0, 1)))

    # Construct the rotation matrix  ( V Transpose(V) - I ) R.

    M = (np.outer(V, V) - np.eye(3)).dot(R)
    return M

def cyclic_permutations(points):
    new_points = []
    for p in points:
        new_points.append(p)
        new_points.append([p[1], p[2], p[0]])
        new_points.append([p[2], p[0], p[1]])
    return new_points


def all_permutations(points):
    new_points = []
    for p in points:
        new_points.extend(cyclic_permutations([p]))
        new_points.extend(cyclic_permutations([[p[1], p[0], p[2]]]))
    return new_points


def plus_minus(points):
    new_points = []
    for p in points:
        mults = []
        for i in range(len(p)):
            if same(p[i], 0):
                mults.append([1])
            else:
                mults.append([1, -1])
        for i in mults[0]:
            for j in mults[1]:
                for k in mults[2]:
                    new_points.append([p[0] * i, p[1] * j, p[2] * k])
    return new_points

def conf1(points):
    return plus_minus(cyclic_permutations(points))


def conf2(points):
    return plus_minus(all_permutations(points))

def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    theta = np.asarray(theta)
    axis = axis / sqrt(np.dot(axis, axis))
    a = cos(theta / 2.0)
    b, c, d = -axis * sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([ \
            [aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)], \
            [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)], \
            [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

def simple_rotate(points, axis, angle):
    rm = rotation_matrix(axis, angle)
    new_points = []
    for p in points:
        new_points.append(np.dot(rm, p))
    return new_points

def rotate1(points):
    parts = 10
    new_points = copy.deepcopy(points)

    shuffled_indices = list(range(len(points)))
    random.shuffle(shuffled_indices)

#     for i in range(len(points)):
    for i in shuffled_indices:
        new_points = copy.deepcopy(points)
        rotated1 = simple_rotate(points[:i], points[i], 2 * pi / parts)
        rotated2 = simple_rotate(points[i+1:], points[i], 2 * pi / parts)
#         new_points.extend(rotated1)
#         new_points.extend(rotated2)
        for p in rotated1 + rotated2:
            if random.random() > 0.2:
                new_points.append(p)
        points = copy.deepcopy(new_points)
        points = uniq_points(points)
        print(len(points))
        if len(points) > 1000:
            break
    return new_points

def distance(a, b):
    return sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2 + (a[2] - b[2]) ** 2)

def sphere_distance(a, b):
    mult = a[0] * b[0] + a[1] * b[1] + a[2] * b[2]
    mult = max(min(mult, 1), -1)
    return math.acos(mult)

def set_radius(points, radius):
    for i in range(len(points)):
        coef = radius / distance(points[i], [0.0, 0.0, 0.0])
        for j in range(len(points[i])):
            points[i][j] = points[i][j] * coef
    #print(points, file=sys.stderr)
    return points

def uniq_points(points, debug=False, eps=LESS_EPS):
    if debug:
        print("before uniq points:", len(points))
    for_remove = set()
    points_tmp = []
    for p in points:
        points_tmp.append(list(p))
        points_tmp.append([-x for x in list(p)])
    points = points_tmp
    points.sort(key=lambda x: x[0])
    for i in range(len(points)):
        if i not in for_remove:
            for j in range(i + 1, len(points)):
                if j not in for_remove and same_points(points[i], points[j], eps):
                    for_remove.add(j)
                if abs(points[j][0] - points[i][0]) >= eps:
                    break
    filtered_points = []
    for i in range(len(points)):
        if i not in for_remove:
            filtered_points.append(points[i])
    if debug:
        print("after uniq points:", len(filtered_points))
    return filtered_points

def add_opposite_points(points, eps=LESS_EPS):
    sorted_points = []
    for p in points:
        if (p[0] > 0) or\
                ((p[0] == 0) and (p[1] > 0)) or\
                ((p[0] == 0) and (p[1] == 0) and (p[2] > 0)):
            sorted_points.append(list(p))

    sorted_points.sort(key=lambda x: -x[0])

    opposite_points = dict()
    points_len = len(sorted_points)
    for i in reversed(range(points_len)):
        sorted_points.append([-x for x in sorted_points[i]])
        j = len(sorted_points) - 1
        opposite_points[i] = j
        opposite_points[j] = i

    return sorted_points, opposite_points

def find_trinities_from_points_old(points, opposite_points, radius):
    # TODO: it's possible to split sphere into 3 parts
    # corresponding to tetrahedron faces
    # each face (without boundary) can only have 1 point from any trinity
    # I guess even better would be to use icosidodecahedron

    # ? FIXME
#     points_with_index = [(p, i) for i, p in enumerate(points)]
#     sorted_points = []
#     for coord in range(3):
#         p_is = copy.copy(points_with_index)
#         p_is.sort(key=lambda x: -x[0][i])
#         sorted_points.append(p_is)

    # ? FIXME
    # NOTE: assuming that points are sorted by first coordinate already
    for i in range(1, len(points)):
        assert (points[i][0] <= points[i - 1][0])

    pairs = dict()
    triangle_distance = radius * sqrt(3)
    for i in range(len(points)):
        pairs[i] = set()
    for i in range(len(points)):
        # ? FIXME
#         p1 = points[i]
#         p1_max_abs_coord_idx
        
        for j in reversed(range(i + 1, len(points))):
            if abs(points[i][0] + points[j][0]) > radius:
                break
            if opposite_points[j] not in pairs[opposite_points[i]] and\
                    opposite_points[i] not in pairs[opposite_points[j]]:
                d = distance(points[i], points[j])
                if same(d, triangle_distance):
                    pairs[i].add(j)
                    opp = [opposite_points[i], opposite_points[j]]
                    opp.sort()
                    pairs[opp[0]].add(opp[1])
    for i in range(len(points)):
        if not pairs[i]:
            pairs.pop(i)

    trinities = set()
    for i in pairs:
        candidates = list(pairs[i])
        candidates.sort()
        for j in range(len(candidates)):
            for k in range(j + 1, len(candidates)):
                if candidates[j] in pairs and candidates[k] in pairs[candidates[j]]:
                    trinities.add((i, candidates[j], candidates[k]))
                    opp = [opposite_points[i], opposite_points[candidates[j]], opposite_points[candidates[k]]]
                    opp.sort()
                    trinities.add(tuple(opp))
    return trinities


def find_trinities_from_points(points, opposite_points, radius, eps=EPS):
    kd_tree = KDTree(points)  # , leaf_size=30)
    triangle_distance = radius * sqrt(3)

    circle_h = -1/2
    circle_r = radius * sqrt(1 - circle_h**2)
    
    reps = max(10, int(sqrt(len(points))/4))

    pairs = defaultdict(set)

    # sample points in circle:
    # circle_p = circle_center + circle_r (v1 cos(t) + v2 sin(t))
    # v1 and v2 are norm 1 and perpendicular to p1
    all_circle_ps = []
    for i, p1 in enumerate(points):
        op1_idx = opposite_points[i]
        if op1_idx < i:
            continue

        v1 = perp1(p1)
        v2 = perp2(p1)
        circle_center = np.array(p1) * circle_h
        t_ang_half = 2 * pi / reps / 2
        circle_p0 = circle_r * (v1 * cos(0) + v2 * sin(0))
        circle_p_half = circle_r * (v1 * cos(t_ang_half) + v2 * sin(t_ang_half))
        search_r = distance(circle_p_half, circle_p0) + eps
        for t_lin in range(reps):
            t_ang = 2 * pi * t_lin / reps
            all_circle_ps.append(circle_center + circle_r * (v1 * cos(t_ang) + v2 * sin(t_ang)))

    all_candidates_lists = list(kd_tree.query_radius(all_circle_ps, search_r))

    point_cnt = -1
    for i, p1 in enumerate(points):
        op1_idx = opposite_points[i]
        if op1_idx < i:
            continue
        point_cnt += 1
        candidates_lists = all_candidates_lists[point_cnt*reps:(point_cnt+1)*reps]
        candidates = set(chain(*candidates_lists))

        for j in candidates:
            p2 = points[j]
            op2_idx = opposite_points[j]
            if op2_idx not in pairs[op1_idx] and op1_idx not in pairs[op2_idx]:
                d = distance(p1, p2)
                if same(d, triangle_distance, eps=eps):
                    pairs[i].add(j)
                    opp = [op1_idx, op2_idx]
                    opp.sort()
                    pairs[opp[0]].add(opp[1])

    trinities = set()
    for i in pairs:
        candidates = list(pairs[i])
        candidates.sort()
        for j in range(len(candidates)):
            for k in range(j + 1, len(candidates)):
                if candidates[k] in pairs[candidates[j]]:
                    trinities.add((i, candidates[j], candidates[k]))
                    opp = [opposite_points[i], opposite_points[candidates[j]], opposite_points[candidates[k]]]
                    opp.sort()
                    trinities.add(tuple(opp))
    return trinities

# filter out uninteresting trinities
def filter_trinities(trinities):
    while True:
        prev_size = len(trinities)
        counts = dict()
        for t in trinities:
            for i in range(len(t)):
                if t[i] not in counts:
                    counts[t[i]] = 0
                counts[t[i]] += 1
        new_trinities = set()
        for t in trinities:
            ones = 0
            for i in range(len(t)):
                if counts[t[i]] == 1:
                    ones += 1
            if ones < 1: # probably don't need a fix, but previously FIXME should be "if ones < 2:"
                new_trinities.add(t)
        trinities = copy.deepcopy(new_trinities)
        if len(trinities) == prev_size:
            break
    return trinities

# map values to smaller
def map_to_smaller_indices(all_points, all_opposite_points, trinities):
    points = []
    smaller_indices = dict()
    cur_idx = 0
    for t in trinities:
        for i in range(len(t)):
            if t[i] not in smaller_indices:
                smaller_indices[t[i]] = cur_idx
                points.append(all_points[t[i]])
                cur_idx += 1

    opposite_points = dict()
    for idx in smaller_indices:
        opposite_points[smaller_indices[idx]] = smaller_indices[all_opposite_points[idx]]

    new_trinities = set()
    for t in trinities:
        new_trinities.add((smaller_indices[t[0]], smaller_indices[t[1]], smaller_indices[t[2]]))

    return points, opposite_points, new_trinities

def get_trinities_by_idx(trinities):
    trinities_by_idx = dict()
    for t in trinities:
        for v in t:
            if v not in trinities_by_idx:
                trinities_by_idx[v] = []
            not_vs = []
            for x in t:
                if x != v:
                    not_vs.append(x)
            trinities_by_idx[v].append(tuple(not_vs))
    return trinities_by_idx

def get_dists(points):
    dists = []
    for p1_idx, p1 in enumerate(points):
        min_dist = None
        for p2_idx, p2 in enumerate(points):
            if p2_idx == p1_idx:
                continue
            cur_dist = sphere_distance(p1, p2)
            if p1_idx == 0:
                has_same_dist = False
                for dist in dists:
                    if abs(cur_dist - dist) < EPS:
                        has_same_dist = True
                        break
                if not has_same_dist:
                    dists.append(cur_dist)
    dists.sort()
    return dists

def prepare_points(points, radius=1, rotate=False, debug=True):
    points = set_radius(points, radius)

    if rotate:
        points_rot_m = rand_rotation_matrix()
        rotated_points = []
        for p in points:
            rotated_points.append(list(np.matmul(p, points_rot_m)))
        points = rotated_points

    if debug:
        print(len(points))
    points = uniq_points(points)
    if debug:
        print(len(points))
    points, opposite_points = add_opposite_points(points)
    if debug:
        print(len(points))
    trinities = find_trinities_from_points(points, opposite_points, radius)
    if debug:
        print(len(points), len(trinities))
    trinities = filter_trinities(trinities)
    if debug:
        print(len(points), len(trinities))
    points, opposite_points, trinities = map_to_smaller_indices(points, opposite_points, trinities)
    if debug:
        print(len(points), len(trinities))
    return points, opposite_points, trinities


In [ ]:
backtrack_from = None
backtrack_to = None
backtrack_parent = dict()
fail_count = defaultdict(int)

def find_colors(cur_idx, max_idx, flooded, color_order0,
                points, opposite_points, trinities_by_idx, colors,
                flow_upper_bound, mod_flow):
    global backtrack_from
    global backtrack_to
    global backtrack_parent
    global fail_count
    if cur_idx > max_idx:
        return True
    cur_val = flooded[cur_idx]
    if colors[cur_val]:
        if find_colors(cur_idx + 1, max_idx, flooded, color_order0,
                       points, opposite_points, trinities_by_idx, colors,
                       flow_upper_bound, mod_flow):
            return True
        else:
            return False
    op_val = opposite_points[cur_val]

    all_color_candidates = set()

    # NOTE: very nice optimization
    forbidden_colors = set()
    for t in trinities_by_idx[cur_val]:
        for v in t:
            forbidden_color = -colors[v]
            if mod_flow:
                forbidden_color %= flow_upper_bound
            forbidden_colors.add(forbidden_color)
            all_color_candidates.add(v)

    color_order = []
    for color in color_order0:
        if abs(color) >= flow_upper_bound:
            continue
        if color not in forbidden_colors:
            color_order.append(color)
    # NOTE: seems like a good optimization
    random.shuffle(color_order)

    backtrack_parent[cur_val] = cur_idx
    backtrack_parent[op_val] = cur_idx
    progressed = False

    for cur_color_idx, cur_color in enumerate(color_order):
        colors[cur_val], colors[op_val] = cur_color, -cur_color
        if mod_flow:
            colors[cur_val] %= flow_upper_bound
            colors[op_val] %= flow_upper_bound
        all_good = True
        also_colored = set()
        to_check = set([cur_val, op_val])
        checked_trinities = set()
        while all_good and to_check:
            val_for_check = to_check.pop()
            for t in trinities_by_idx[val_for_check]:
                if t in checked_trinities:  # FIXME: probably no need for this structure
                    continue
                checked_trinities.add(t)
                zeros_count = 0
                zeros_vals = []
                for v in t:
                    if not colors[v]:
                        zeros_count += 1
                        zeros_vals.append(v)
                if zeros_count == 2:
                    continue
                for v in t:
                    all_color_candidates.add(v)
                flow = colors[val_for_check] + colors[t[0]] + colors[t[1]]
                if mod_flow:
                    flow %= flow_upper_bound
                if zeros_count == 0:
                    all_good = (flow == 0)
                else:
                    all_good = (flow != 0) and (abs(flow) < flow_upper_bound)
                if not all_good:
                    break
                if zeros_count == 1:
                    v = zeros_vals[0]
                    op_v = opposite_points[v]
                    colors[v], colors[op_v] = -flow, flow
                    if mod_flow:
                        colors[v] %= flow_upper_bound
                        colors[op_v] %= flow_upper_bound
                    also_colored.add(v)
                    also_colored.add(op_v)
                    to_check.add(v)
                    to_check.add(op_v)
                    backtrack_parent[v] = cur_idx
                    backtrack_parent[op_v] = cur_idx
        if all_good:
            progressed = True
            if find_colors(cur_idx + 1, max_idx, flooded, color_order0,
                           points, opposite_points, trinities_by_idx, colors,
                           flow_upper_bound, mod_flow):
                return True
        for idx in also_colored:
            colors[idx] = 0
            backtrack_parent.pop(idx)
        if backtrack_from is not None:
            assert (backtrack_to <= cur_idx)
            if backtrack_to < cur_idx:
                break
            else:
                if cur_color_idx == len(color_order) - 1:
                    flooded[backtrack_to], flooded[backtrack_from] =\
                        flooded[backtrack_from], flooded[backtrack_to]
                else:
                    flooded[backtrack_to + 1], flooded[backtrack_from] =\
                        flooded[backtrack_from], flooded[backtrack_to + 1]
                backtrack_to = None
                backtrack_from = None

    colors[cur_val], colors[op_val] = 0, 0
    backtrack_parent.pop(cur_val)
    backtrack_parent.pop(op_val)

    if backtrack_from is None and not progressed:
        backtrack_to = None
        for v in all_color_candidates:
            if v in backtrack_parent:
                if backtrack_to is None:
                    backtrack_to = 0
                backtrack_to = max(backtrack_to, backtrack_parent[v])
                assert (backtrack_to <= cur_idx)
        assert (backtrack_to is not None)
        backtrack_from = cur_idx
        assert (backtrack_to <= backtrack_from)
        assert (backtrack_to <= cur_idx)

    if not progressed:
        fail_count[cur_val] += 1

    if backtrack_from is None:
        next_val = flooded[cur_idx + 1]
        if fail_count[next_val] > fail_count[cur_val]:
            flooded[cur_idx] = next_val
            flooded[cur_idx + 1] = cur_val
            fail_count[next_val] -= 1

    return False


In [ ]:
def find_flow(points, opposite_points, trinities, flow_values, mod_flow=False, seed=-1):
    if seed == -1:
        seed = random.randint(0, 1000000)
        print('seed', seed)
    random.seed(seed)
    global backtrack_to
    global backtrack_from
    global backtrack_parent
    global fail_count

    colors = dict()
    for i in opposite_points:
        colors[i] = 0
    trinities_by_idx = get_trinities_by_idx(trinities)

    found5 = True
    component_count = 0
    
    shuffled_indices = list(range(len(points)))
    random.shuffle(shuffled_indices)
    
    for prioritize_huge_trinities in [True, False]:
        if not found5:
            break
        for seed_vertex in shuffled_indices:
            is_huge_trinity = True
            if prioritize_huge_trinities:
                is_huge_trinity = (len(trinities_by_idx[seed_vertex]) > 2)
#                 and (len(trinities_by_idx[seed_vertex]) < 10)
            if (not colors[seed_vertex]) and is_huge_trinity:
                component_count += 1
                flooded = []
                should_check = set([seed_vertex])
                to_check = set([seed_vertex])
                checked = set()
                while to_check:
                    cur_val = to_check.pop()
                    if cur_val not in flooded:
                        flooded.append(cur_val)
                    checked.add(cur_val)

                    done_1_flood = False
                    neibs = []
                    for t in trinities_by_idx[cur_val]:
                        for v in t:
                            neibs.append((-len(trinities_by_idx[v]), v))

                    random.shuffle(neibs)
#                     neibs.sort()

    #                 random.shuffle(trinities_by_idx[cur_val])
    #                 trinities_by_idx[cur_val].sort()
    #                 for t in trinities_by_idx[cur_val]:
    #                     t = list(t)
    #                     random.shuffle(t)
                    for _, v in neibs:
                        if v in flooded:
                            continue
                        if not done_1_flood:
                            flooded.append(v)
                            done_1_flood = True
                        to_check.add(v)
                        should_check.add(v)
                    if not to_check:
                        to_check = should_check - checked
                max_idx = len(flooded) - 1

                for flow_upper_bound in flow_values:
                    color_order = []
                    for i in range(1, flow_upper_bound):
                        color_order.append(i)
                        if not mod_flow:
                            color_order.append(-i)
                    for v in flooded:
                        colors[v] = 0
                    
                    backtrack_to = None
                    backtrack_from = None
                    backtrack_parent = dict()
                    fail_count = defaultdict(int)

                    cur_found5 = find_colors(0, max_idx, flooded, color_order,
                                             points, opposite_points, trinities_by_idx, colors,
                                             flow_upper_bound, mod_flow)
                    print(len(flooded), 'cur_found:', flow_upper_bound, cur_found5)
                    if cur_found5 and mod_flow:
                        
                        color_order = []
                        for i in range(1, flow_upper_bound):
                            color_order.append(i)
                            color_order.append(-i)
                        colors_alt = dict()
                        for i in opposite_points:
                            colors_alt[i] = 0
                        for v in flooded:
                            colors_alt[v] = 0

                        backtrack_to = None
                        backtrack_from = None
                        backtrack_parent = dict()
                        fail_count = defaultdict(int)

                        cur_found5_alt = find_colors(0, max_idx, flooded, color_order,
                                                     points, opposite_points, trinities_by_idx, colors_alt,
                                                     flow_upper_bound, False)
                        print('alt', 'cur_found:', flow_upper_bound, cur_found5_alt)
                    if cur_found5:
                        break
                found5 = found5 and cur_found5
                if not found5:
                    break

                flows5 = dict()
                for v in should_check:
                    flows5[v] = colors[v]
                print(len(flooded), 'vs', len(points))
        #             print("color", colors[v], sep='\t')
        #         if len(flooded) == len(points):
        #             for t in trinities:
        #                 print("trinity", t[0], t[1], t[2], sep='\t')
        #         print("flow values:", flows5)

    print('found', flow_values, found5)
    print('component_count', component_count)

    return colors


In [49]:
def gen_icosidodecahedron():
    points = []
    phi = (1.0 + sqrt(5)) / 2
    a, b, c = 0.5, phi / 2, (1.0 + phi) / 2
#     a, b, c = 1, phi, (1.0 + phi)
    points.extend(conf1([[a, b, c]]))
    points.extend(conf1([[0.0, 0.0, 1.0]]))
#     points = rotate1(points)
    return points

def gen_more():
    points = []
    phi = (1.0 + sqrt(5)) / 2
#     a, b, c = 0.5, phi / 2, (1.0 + phi) / 2
    a, b, c = 1, phi, (1.0 + phi)
    points.extend(conf2([[a, b, c]]))
    points.extend(conf2([[0.0, 0.0, b]]))
    points.extend(conf2([[0.0, a, b]]))
    points.extend(conf2([[0.0, a, c]]))

    points.extend(conf2([[0.0, 1.0, 1.0]]))

    for k in [3]:
        phi = (1.0 + sqrt(k)) / 2
        a, b, c, d = 1, phi, phi**2, (1.0 + phi)
        points.extend(conf2([[0.0, 1.0, b]]))
        points.extend(conf2([[0.0, 1.0, c]]))
        points.extend(conf2([[0.0, 1.0, d]]))
        points.extend(conf2([[0.0, a, b]]))
        points.extend(conf2([[0.0, a, c]]))
        points.extend(conf2([[0.0, a, d]]))
        points.extend(conf2([[0.0, b, d]]))
        points.extend(conf2([[0.0, c, d]]))
        points.extend(conf2([[a, b, c]]))
        points.extend(conf2([[a, b, d]]))
        points.extend(conf2([[a, c, d]]))
        points.extend(conf2([[b, c, d]]))

    return points

# points = gen_more()
# phi = (1 + 5**0.5)/2
# radius = 2 * phi
# points, opposite_points, trinities = prepare_points(points, radius=radius, rotate=True)

points = gen_icosidodecahedron()
radius = 1
points, opposite_points, trinities = prepare_points(points, radius=radius, rotate=False)

# dists = get_dists(points)
# print('dists')
# for dist in dists:
#     print(dist)

magic_dist = 1.2566370614359175 * radius

30
30
30
30 20
30 20
30 20


In [50]:
colors = find_flow(points, opposite_points, trinities, flow_values=[4, 5], mod_flow=True)

seed 924146
30 cur_found: 4 False
30 cur_found: 5 True
alt cur_found: 5 True
30 vs 30
found [4, 5] True
component_count 1


In [51]:
# ratios1 = [0, 1/2, 1/3, 2/3, 1/4, 3/4, 1/5, 2/5, 3/5, 4/5, 1/6, 5/6]
# ratios2 = [1/8, 3/8, 5/8, 7/8]

all_ratios = [
              '0',
              '1/2',
              '1/3', '2/3',
              '1/4', '3/4',
              '1/5', '2/5', '3/5', '4/5',
              '1/6', '5/6'
             ]

all_ratios3 = [
              '0',
              '1/3', '2/3',
#               '1/2',
#               '1/6', '5/6'
             ]

all_ratios5 = [
              '0',
              '1/5', '2/5', '3/5', '4/5',
             ]


In [52]:
def expand_points(points, opposite_points, ratios, radius=1,
                  dists=None, prop=1.0, seed=42):
#     assert (radius == 1)  # FIXME: fix the code to work with other radius values
    if seed == -1:
        seed = random.randint(0, 1000000)
    print('seed', seed)
    random.seed(seed)
    circle = []
    for p1_idx, p1 in enumerate(points):
        circle.append(copy.copy(p1))

    for h1_str in ratios:
        for h2_str in ratios:
            h1 = eval(h1_str)
            h2 = eval(h2_str)
            if (h2 < h1):
                continue

            r = radius * sqrt(1 - h1**2)

            for p1_idx, p1 in enumerate(points):
                if (h1 == 0) and (opposite_points[p1_idx] < p1_idx):
                    continue
                for p2_idx, p2 in enumerate(points):
                    if p2_idx == p1_idx:
                        continue
                    if (h1 == h2) and (p2_idx < p1_idx):
                        continue
                    if (h2 == 0) and (opposite_points[p2_idx] < p2_idx):
                        continue
                    if opposite_points[p1_idx] == p2_idx:
                        continue
                    if dists is not None:
                        dist = sphere_distance(p1, p2)
                        if dist <= dists[0] - EPS:
                            continue
                        if dist >= dists[-1] + EPS:
                            continue
                    
                    if random.random() > prop:
                        continue

                    # 1. intersect planes
                    # get line v = a + tb
                    p1p2 = np.dot(p1, p2)
                    denom = 1 - p1p2**2
                    c1 = (h1 - h2 * p1p2) / denom
                    c2 = (h2 - h1 * p1p2) / denom
                    a = c1 * np.array(p1) + c2 * np.array(p2)
                    b = np.cross(p1, p2)

                    # 2. intersect line and circle
                    # circle:
                    # circle_p = circle_center + r (u cos(t) + v sin(t))
                    # u and v are norm 1 and perpendicular to p1
                    circle_center = np.array(p1) * h1
                    line_dir_vec = np.array(normed(b))
                    scant = circle_center - a
                    t = np.dot(scant, line_dir_vec)
                    proj = a + t * line_dir_vec
                    dist_to_line = distance(proj, circle_center)
                    if dist_to_line > r:
                        continue
                    for line_dir_sign in [1, -1]:
                        if (line_dir_sign == -1) and same(dist_to_line, r):
                            break
                        circle_p = proj + line_dir_sign * line_dir_vec * sqrt(r**2 - dist_to_line**2)
                        circle_p = np.array(circle_p)
                        circle.append(circle_p)
                        circle.append(normed(-np.array(p1) - circle_p))
                        circle.append(normed(-np.array(p2) - circle_p))

    points2, opposite_points2, trinities2 = prepare_points(circle, radius=radius)

    if len(trinities2) > 20:
        trinities_by_idx2 = get_trinities_by_idx(trinities2)

        test_colors = dict()
        for seed_vertex in opposite_points:
            test_colors[seed_vertex] = 0

        component_count = 0
        max_flooded_len = 0
        for seed_vertex in opposite_points:
            if test_colors[seed_vertex] == 0:
                component_count += 1
                flooded = []
                should_check = set([seed_vertex])
                to_check = set([seed_vertex])
                checked = set()
                while to_check:
                    cur_val = to_check.pop()
                    test_colors[cur_val] = 1
                    if cur_val not in flooded:
                        flooded.append(cur_val)
                    checked.add(cur_val)
                    done_1_flood = False
                    if cur_val in trinities_by_idx2:
                        random.shuffle(trinities_by_idx2[cur_val])
                        for t in trinities_by_idx2[cur_val]:
                            for v in t:
                                if v not in flooded:
                                    if not done_1_flood:
                                        flooded.append(v)
                                        done_1_flood = True
                                    to_check.add(v)
                                    should_check.add(v)
                    if not to_check:
                        to_check = should_check - checked
                max_flooded_len = max(len(flooded), max_flooded_len)

        if max_flooded_len > 30:
            print(ratios,
                  'max_flooded_len', max_flooded_len,
                  'len', len(trinities2), component_count)

    print('done')
    return points2, opposite_points2, trinities2

In [56]:
points2, opposite_points2, trinities2 = expand_points(points, opposite_points,
                                                      ['1/2'],
                                                      prop=0.4,
                                                      dists=[magic_dist],
                                                      seed=420163)

seed 420163
180
294
294
294 84
294 40
50 40
['1/2'] max_flooded_len 50 len 40 1
done


In [57]:
# phi = (1+5**0.5)/2
# new_radius = 2 * phi

new_radius = 1

for p in points2:
    print([x*new_radius for x in p])

[0.8090169943749476, -0.30901699437494745, -0.5]
[0.0, 0.0, 1.0]
[-0.8090169943749476, 0.30901699437494745, -0.5]
[0.5, 0.8090169943749476, 0.30901699437494745]
[0.47775729935136957, -0.7223212324973467, -0.5000000000000001]
[-0.9777572993513695, -0.08669576187760075, 0.1909830056250526]
[-0.47775729935136957, 0.7223212324973467, -0.5000000000000001]
[0.6687403049764221, -0.41330423812239936, 0.6180339887498949]
[-0.30901699437494745, -0.5, -0.8090169943749476]
[-0.3597233106014746, 0.9133042381223995, 0.19098300562505266]
[0.5, -0.8090169943749476, -0.30901699437494745]
[-1.0, -0.0, -0.0]
[0.8090169943749476, -0.30901699437494745, 0.5]
[-0.5, 0.8090169943749476, 0.30901699437494745]
[0.3597233106014746, -0.9133042381223995, -0.19098300562505266]
[0.30901699437494745, 0.5, 0.8090169943749476]
[-0.6687403049764221, 0.41330423812239936, -0.6180339887498949]
[0.8597233106014748, -0.10428724374745182, 0.5000000000000001]
[-0.0, -0.0, -1.0]
[-0.8597233106014748, 0.10428724374745182, 0.50000

In [60]:
colors = find_flow(points2, opposite_points2, trinities2, flow_values=[4, 5, 6], mod_flow=True)

seed 267475
50 cur_found: 4 False
50 cur_found: 5 False
50 cur_found: 6 True
alt cur_found: 6 True
50 vs 50
found [4, 5, 6] True
component_count 1


In [105]:
points3, opposite_points3, trinities3 = expand_points(points2, opposite_points2,
                                                      ['1/8'],
#                                                       prop=0.7,
#                                                       dists=[magic_dist],
                                                      seed=-1)

seed 632271
7130
7130
7130
7130 320
7130 140
170 140
['1/8'] max_flooded_len 170 len 140 1
done


In [107]:
colors = find_flow(points3, opposite_points3, trinities3, flow_values=[4, 5, 6], mod_flow=True)

seed 153713
170 cur_found: 4 False
170 cur_found: 5 False
170 cur_found: 6 True
alt cur_found: 6 True
170 vs 170
found [4, 5, 6] True
component_count 1


In [114]:
points4, opposite_points4, trinities4 = expand_points(points3, opposite_points3,
                                                      ['1/2'],
                                                      prop=0.3,
#                                                       dists=[magic_dist],
                                                      seed=-1)

seed 289046
19949
30712
30712
30712 2002
30712 564
584 564
['1/2'] max_flooded_len 584 len 564 1
done


In [119]:
colors = find_flow(points4, opposite_points4, trinities4, flow_values=[4, 5, 6], mod_flow=True)

seed 723186
584 cur_found: 4 False
584 cur_found: 5 False
584 cur_found: 6 True
alt cur_found: 6 True
584 vs 584
found [4, 5, 6] True
component_count 1


In [94]:
# 0 1/2 max_flooded_len 270 len 300 1
# 0 3/5 max_flooded_len 150 len 140 1
# 1/2 1/2 max_flooded_len 510 len 500 1
# 1/2 3/4 max_flooded_len 150 len 120 1
# 1/4 1/2 max_flooded_len 150 len 160 3
# 1/4 1/4 max_flooded_len 150 len 160 3
# 1/4 3/4 max_flooded_len 150 len 160 2
# 3/5 4/5 max_flooded_len 60 len 50 1
# 1/6 2/3 max_flooded_len 270 len 200 1

# 0, 1/3, 2/3
# max_flooded_len 270 len 240 1

# 0, 1/5, 2/5, 3/5, 4/5
# max_flooded_len 750 len 780 1

# 0, 1/6, 1/3, 1/2, 2/3, 5/6
# max_flooded_len 1470 len 1440 1

In [116]:

# 1/2, 1/4, 3/4
# 3/5, 0
# 3/5, 4/5

circle = []
for p1_idx, p1 in enumerate(points):
    circle.append(copy.copy(p1))

for h1, h2 in [
#                (0, 1/2),
               (1/2, 1/2),  # breaks nz5flow, contains nz6flow
#                (1/2, 3/4),
#                (1/2, 1/4), (1/4, 1/4), (1/4, 3/4),
#                (0, 3/5),
#                (3/5, 4/5),
#                (1/6, 2/3),
              ]:


    r = sqrt(1 - h1**2)

    for p1_idx, p1 in enumerate(points):
        if (h1 == 0) and (opposite_points[p1_idx] < p1_idx):
            continue
        for p2_idx, p2 in enumerate(points):
            if p2_idx == p1_idx:
                continue
            if (h1 == h2) and (p2_idx < p1_idx):
                continue
            if (h2 == 0) and (opposite_points[p2_idx] < p2_idx):
                continue
            if opposite_points[p1_idx] == p2_idx:
                continue

            if abs(sphere_distance(p1, p2) - dists[2]) >= EPS:
                continue

            if random.random() > 0.4:
                continue

            assert (radius == 1)
            # 1. intersect planes
            # get line v = a + tb
            p1p2 = np.dot(p1, p2)
            denom = 1 - p1p2**2
            c1 = (h1 - h2 * p1p2) / denom
            c2 = (h2 - h1 * p1p2) / denom
            a = c1 * np.array(p1) + c2 * np.array(p2)
            b = np.cross(p1, p2)

            # 2. intersect line and circle
            # circle:
            # circle_p = circle_center + r (u cos(t) + v sin(t))
            # u and v are norm 1 and perpendicular to p1
            circle_center = np.array(p1) * h1
            line_dir_vec = np.array(normed(b))
            scant = circle_center - a
            t = np.dot(scant, line_dir_vec)
            proj = a + t * line_dir_vec
            dist_to_line = distance(proj, circle_center)
            if dist_to_line > r:
                continue
            for line_dir_sign in [1, -1]:
                if (line_dir_sign == -1) and same(dist_to_line, r):
                    break
                circle_p = proj + line_dir_sign * line_dir_vec * sqrt(r**2 - dist_to_line**2)
                circle_p = np.array(normed(circle_p))
                circle.append(circle_p)
                circle.append(normed(-np.array(p1) - circle_p))
                circle.append(normed(-np.array(p2) - circle_p))

print(len(circle))
points2 = set_radius(circle, radius)
points2 = uniq_points(points2)
points2, opposite_points2 = add_opposite_points(points2)
print(len(points2))
trinities2 = find_trinities_from_points(points2, opposite_points2, radius)
print(len(points2), len(trinities2))
trinities2 = filter_trinities(trinities2)
print(len(points2), len(trinities2))
points2, opposite_points2, trinities2 = map_to_smaller_indices(points2, opposite_points2, trinities2)
print(len(points2), len(trinities2))

if len(trinities2) > 20:
    trinities_by_idx2 = dict()
    for t in trinities2:
        for v in t:
            if v not in trinities_by_idx2:
                trinities_by_idx2[v] = []
            not_vs = []
            for x in t:
                if x != v:
                    not_vs.append(x)
            trinities_by_idx2[v].append(tuple(not_vs))

    test_colors = dict()
    for seed_vertex in opposite_points:
        test_colors[seed_vertex] = 0

    component_count = 0
    max_flooded_len = 0
    for seed_vertex in opposite_points:
        if test_colors[seed_vertex] == 0:
            component_count += 1
            flooded = []
            should_check = set([seed_vertex])
            to_check = set([seed_vertex])
            checked = set()
            while to_check:
                cur_val = to_check.pop()
                test_colors[cur_val] = 1
                if cur_val not in flooded:
                    flooded.append(cur_val)
                checked.add(cur_val)
                done_1_flood = False
                if cur_val in trinities_by_idx2:
                    random.shuffle(trinities_by_idx2[cur_val])
                    for t in trinities_by_idx2[cur_val]:
                        for v in t:
                            if v not in flooded:
                                if not done_1_flood:
                                    flooded.append(v)
                                    done_1_flood = True
                                to_check.add(v)
                                should_check.add(v)
                if not to_check:
                    to_check = should_check - checked
            max_flooded_len = max(len(flooded), max_flooded_len)

    if max_flooded_len > 30:
        print(h1, h2, 'max_flooded_len', max_flooded_len,
              'len', len(trinities2), component_count)

print('done')
# print('done', h1, h2)

168
258
258 68
258 40
50 40
0.5 0.5 max_flooded_len 50 len 40 1
done


In [117]:
# 1/2, 1/4, 3/4
# 3/5, 0
# 3/5, 4/5

In [118]:
# dist_EPS = EPS * 0.9

# profiles2 = []
# profile2_counts = defaultdict(int)
# all_dists2 = []
# for p1_idx, p1 in enumerate(points2):
#     cur_dists2 = []
#     cur_dists2_counts = defaultdict(int)
#     for p2_idx, p2 in enumerate(points2):
#         if p2_idx == p1_idx:
#             continue
#         cur_dist2 = sphere_distance(p1, p2)
#         has_same_dist2 = False
#         for dist2 in cur_dists2:
#             if abs(cur_dist2 - dist2) < dist_EPS:
#                 has_same_dist2 = True
#                 cur_dists2_counts[dist2] += 1
#                 break
#         if not has_same_dist2:
#             cur_dists2.append(cur_dist2)
#             cur_dists2_counts[cur_dist2] = 1
#     cur_dists2 = list(cur_dists2_counts.items())
#     cur_dists2.sort()

#     found_profile_idx = None
#     for dist2_profile_idx, dist2_profile in enumerate(all_dists2):
#         if len(dist2_profile) != len(cur_dists2):
#             continue
#         all_same = True
#         for dist2_idx in range(len(dist2_profile)):
#             if abs(dist2_profile[dist2_idx][0] - cur_dists2[dist2_idx][0]) >= dist_EPS:
#                 all_same = False
#                 break
#             if dist2_profile[dist2_idx][1] != cur_dists2[dist2_idx][1]:
#                 all_same = False
#                 break
#         if all_same:
#             profiles2.append(dist2_profile_idx)
#             found_profile_idx = dist2_profile_idx
#             break
#     if found_profile_idx is None:
#         all_dists2.append(copy.copy(cur_dists2))
#         profiles2.append(len(all_dists2) - 1)
#     profile2_counts[profiles2[p1_idx]] += 1

# print(len(all_dists2), profile2_counts)
# # for cur_dist2 in all_dists2:
# #     print(cur_dist2)

In [119]:
points2 = set_radius(circle, radius)
points2 = uniq_points(points2)
points2, opposite_points2 = add_opposite_points(points2)
print(len(points2))
trinities2 = find_trinities_from_points(points2, opposite_points2, radius)
print(len(points2), len(trinities2))
trinities2 = filter_trinities(trinities2)
print(len(points2), len(trinities2))
points2, opposite_points2, trinities2 = map_to_smaller_indices(points2, opposite_points2, trinities2)
print(len(points2), len(trinities2))


258
258 68
258 40
50 40


In [120]:
# def get_edges_from_trinities(trinities, opposite_points):
#     # v1,v2 - opposite
#     # trinity => vertex
#     edges = defaultdict(list)
#     t_idx = -1
#     parsed_trinities = set()
#     for t in trinities:
#         normed_trinity = []
#         for v in t:
#             if opposite_points[v] < v:
#                 v = opposite_points[v]
#             normed_trinity.append(v)
#         normed_trinity = tuple(sorted(list(normed_trinity)))
#         if normed_trinity in parsed_trinities:
#             continue
#         parsed_trinities.add(normed_trinity)
#         t_idx += 1
#         for v in normed_trinity:
#             edges[v].append(t_idx)
#     assert (len(parsed_trinities) == len(trinities) // 2)
#     print(edges)
#     return list(edges.values())

# trinities_by_idx2 = get_trinities_by_idx(trinities2)
# for v in trinities_by_idx2:
#     print(v, trinities_by_idx2[v])
# #     assert (len(trinities_by_idx2[v]) == 2)

# from graphviz import Graph

# def new_graph(n, rad=1):
#     dot = Graph(engine='neato')
#     for i in range(n):
# #         ang = 2 * math.pi * i / n
# #         x = math.sin(ang) * rad
# #         y = math.cos(ang) * rad
#         dot.node(str(i), shape='circle', width='0.3', fixedsize='True')#, pos=f'{x},{y}!')
#     return dot

# def add_edges(edges, colors):
#     for edge in edges:
#         assert len(edge) == 2
#         dot.edge(str(edge[0]), str(edge[1]), color=colors[0], penwidth='3')

# graphviz_colors = ['green','red', 'blue', 'purple', 'cyan', 'yellow']
# dot = new_graph(len(trinities2)//2)
# edges = get_edges_from_trinities(trinities2, opposite_points2)
# add_edges(edges, graphviz_colors)
# dot

In [26]:
# plotting points
print(len(points2))
xs = [point[0] for point in points2]
ys = [point[1] for point in points2]
zs = [point[2] for point in points2]

trace = go.Scatter3d(x=xs, y=ys, z=zs, mode='markers',
    marker={
        'size': 10,
        'opacity': 0.8,
        'color': zs,
        'colorscale': 'Viridis',
    },
)
data = [trace]
layout = go.Layout(
    showlegend=True,
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
)
plot_figure = go.Figure(data=data, layout=layout)

# Render the plot.
plotly.offline.iplot(plot_figure)


50


In [122]:

colors = find_flow(points2, opposite_points2, trinities2, flow_values=[4, 5, 6], mod_flow=True)
# colors = find_flow(points2, opposite_points2, trinities2, flow_values=[6], mod_flow=True)
# colors = find_flow(points, opposite_points, trinities, flow_values=[4, 5], mod_flow=True)

50 cur_found: 4 False
50 cur_found: 5 False
50 cur_found: 6 True
alt cur_found: 6 True
50 vs 50
found [4, 5, 6] True
component_count 1


In [441]:
h_scaled = 1/2
h1 = h2 = -h_scaled
r = sqrt(1 - h1**2)
circle = []
for p1_idx, p1 in enumerate(points2):
    if opposite_points2[p1_idx] < p1_idx:
        continue
    circle.append(copy.copy(p1))
    for p2_idx, p2 in enumerate(points2):
        if p2_idx <= p1_idx:
            continue
        if opposite_points2[p2_idx] < p2_idx:
            continue
        if opposite_points2[p1_idx] == p2_idx:
            continue

        if sphere_distance(p1, p2) >= dists[2] + EPS:
            continue
        if sphere_distance(p1, p2) <= dists[2] - EPS:
            continue

#         if abs(sphere_distance(p1, p2) - dists[2]) >= EPS:
#             continue

#         if abs(sphere_distance(p1, p2) - dists[2]) >= EPS and\
#                 abs(sphere_distance(p1, p2) - dists[3]) >= EPS and\
#                 abs(sphere_distance(p1, p2) - dists[1]) >= EPS and\
#                 abs(sphere_distance(p1, p2) - dists[0]) >= EPS:
#             continue

        # 1. intersect planes
        # get line v = a + tb
        p1p2 = np.dot(p1, p2)
        denom = 1 - p1p2**2
        h1 = h2 = -h_scaled
        c1 = (h1 - h2 * p1p2) / denom
        c2 = c1
        a = c1 * np.array(p1) + c2 * np.array(p2)
        b = np.cross(p1, p2)

        # 2. intersect line and circle
        # circle:
        # circle_p = circle_center + r (u cos(t) + v sin(t))
        # u and v are norm 1 and perpendicular to p1
        circle_center = np.array(p1) * h1
        r = sqrt(1 - h1**2)

        line_dir_vec = np.array(normed(b))
        scant = circle_center - a
        t = np.dot(scant, line_dir_vec)
        proj = a + t * line_dir_vec
        dist_to_line = distance(proj, circle_center)
        if dist_to_line > r:
            continue
        circle_p1 = proj + line_dir_vec * sqrt(r**2 - dist_to_line**2)
        circle_p1 = np.array(normed(circle_p1))
        circle.append(circle_p1)
        circle.append(-np.array(p1)-circle_p1)
        circle.append(-np.array(p2)-circle_p1)
        if not same(dist_to_line, r):
            circle_p2 = proj - line_dir_vec * sqrt(r**2 - dist_to_line**2)
            circle_p2 = np.array(normed(circle_p2))
            circle.append(circle_p2)
            circle.append(-np.array(p1)-circle_p2)
            circle.append(-np.array(p2)-circle_p2)

print(len(circle))
points3 = set_radius(circle, radius)
points3 = uniq_points(points3)
points3, opposite_points3 = add_opposite_points(points3)
print(len(points3))
trinities3 = find_trinities_from_points(points3, opposite_points3, radius)
print(len(points3), len(trinities3))
trinities3 = filter_trinities(trinities3)
print(len(points3), len(trinities3))
points3, opposite_points3, trinities3 = map_to_smaller_indices(points3, opposite_points3, trinities3)
print(len(points3), len(trinities3))

# 24777
# 19662
# 19662 14156
# 19662 2000
# 1934 2000

# 49083
# 43338
# 43338 30820
# 43338 4178
# 3972 4178

2259
3314
3314 2676
3314 1476
1514 1476


In [443]:
import sys
sys.setrecursionlimit(10000)
colors = find_flow(points3, opposite_points3, trinities3, flow_values=[5, 6], mod_flow=True)
# for i in range(10):
#     colors = find_flow(points3, opposite_points3, trinities3, flow_values=[6], mod_flow=True)

# colors = find_flow(points3, opposite_points3, trinities3, flow_values=[7], mod_flow=True)
# colors = find_flow(points3, opposite_points3, trinities3, flow_values=[8], mod_flow=True)
# colors = find_flow(points3, opposite_points3, trinities3, flow_values=[9], mod_flow=True)
# colors = find_flow(points3, opposite_points3, trinities3, flow_values=[9])

1490 cur_found: 5 False
1490 cur_found: 6 True


KeyboardInterrupt: 

In [180]:
all_ratios = [
              '0',
#               '1/2',
              '1/3', '2/3',
#               '1/4', '3/4',
#               '1/5', '2/5', '3/5', '4/5',
#               '1/6', '5/6'
             ]

circle3 = []
for p1_idx, p1 in enumerate(points2):
    circle3.append(copy.copy(p1))

for h1_str in all_ratios:
    for h2_str in all_ratios:
        h1 = eval(h1_str)
        h2 = eval(h2_str)

        if (h2 < h1):
            continue

        r = sqrt(1 - h1**2)

        for p1_idx, p1 in enumerate(points2):
            if (h1 == 0) and (opposite_points2[p1_idx] < p1_idx):
                continue
            for p2_idx, p2 in enumerate(points2):
                if p2_idx == p1_idx:
                    continue
                if (h1 == h2) and (p2_idx < p1_idx):
                    continue
                if (h2 == 0) and (opposite_points2[p2_idx] < p2_idx):
                    continue
                if opposite_points2[p1_idx] == p2_idx:
                    continue

                if random.random() > 0.5:
                    continue

                assert (radius == 1)

                # 1. intersect planes
                # get line v = a + tb
                p1p2 = np.dot(p1, p2)
                denom = 1 - p1p2**2
                c1 = (h1 - h2 * p1p2) / denom
                c2 = (h2 - h1 * p1p2) / denom
                a = c1 * np.array(p1) + c2 * np.array(p2)
                b = np.cross(p1, p2)

                # 2. intersect line and circle
                # circle:
                # circle_p = circle_center + r (u cos(t) + v sin(t))
                # u and v are norm 1 and perpendicular to p1
                circle_center = np.array(p1) * h1
                line_dir_vec = np.array(normed(b))
                scant = circle_center - a
                t = np.dot(scant, line_dir_vec)
                proj = a + t * line_dir_vec
                dist_to_line = distance(proj, circle_center)
                if dist_to_line > r:
                    continue
                for line_dir_sign in [1, -1]:
                    if (line_dir_sign == -1) and same(dist_to_line, r):
                        break
                    circle_p = proj + line_dir_sign * line_dir_vec * sqrt(r**2 - dist_to_line**2)
                    circle_p = np.array(normed(circle_p))
                    circle3.append(circle_p)
                    circle3.append(normed(-np.array(p1) - circle_p))
                    circle3.append(normed(-np.array(p2) - circle_p))

print(len(circle3))
points3 = set_radius(circle3, radius)
points3 = uniq_points(points3)
points3, opposite_points3 = add_opposite_points(points3)
print(len(points3))
trinities3 = find_trinities_from_points(points3, opposite_points3, radius)
print(len(points3), len(trinities3))
trinities3 = filter_trinities(trinities3)
print(len(points3), len(trinities3))
points3, opposite_points3, trinities3 = map_to_smaller_indices(points3, opposite_points3, trinities3)
print(len(points3), len(trinities3))
print('done', h1_str, h2_str)

17516
23060
23060 238
23060 74
98 74
done 2/3 2/3


In [182]:
# import sys
# sys.setrecursionlimit(10000)
colors = find_flow(points3, opposite_points3, trinities3, flow_values=[5, 6], mod_flow=True)


74 cur_found: 5 False
74 cur_found: 6 True
alt cur_found: 6 True
74 vs 98
12 cur_found: 5 True
alt cur_found: 5 True
12 vs 98
12 cur_found: 5 True
alt cur_found: 5 True
12 vs 98
found [5, 6] True
component_count 3


In [ ]:
print(dists)

In [439]:
dists3 = []
for p1_idx, p1 in enumerate(points3):
    if p1_idx % 10 == 0:
        print(p1_idx, len(points3), len(dists3))
    for p2_idx in range(p1_idx + 1, len(points3)):
        p2 = points3[p2_idx]
        cur_dist3 = sphere_distance(p1, p2)
#         if cur_dist3 > 0.62:
#             continue
        if cur_dist3 >= dists[3] + EPS:
            continue
        if cur_dist3 <= dists[1] - EPS:
            continue
        has_same_dist3 = False
        for dist3 in dists3:
            if abs(cur_dist3 - dist3) < EPS:
                has_same_dist3 = True
                break
        if not has_same_dist3:
            dists3.append(cur_dist3)
dists3.sort()
for dist3 in dists3:
    print(dist3)

0 1494 0
10 1494 1551
20 1494 1970
30 1494 2131
40 1494 2272
50 1494 2306
60 1494 2358
70 1494 2389
80 1494 2435
90 1494 2436
100 1494 2437
110 1494 2445
120 1494 2458
130 1494 2467
140 1494 2467
150 1494 2472
160 1494 2472
170 1494 2473
180 1494 2479
190 1494 2479
200 1494 2480
210 1494 2480
220 1494 2480
230 1494 2483
240 1494 2483
250 1494 2483
260 1494 2483
270 1494 2483
280 1494 2483
290 1494 2483
300 1494 2483
310 1494 2483
320 1494 2483
330 1494 2483
340 1494 2483
350 1494 2483
360 1494 2483
370 1494 2483
380 1494 2483
390 1494 2483
400 1494 2483
410 1494 2483
420 1494 2483
430 1494 2483
440 1494 2483
450 1494 2483
460 1494 2483
470 1494 2483
480 1494 2483
490 1494 2483
500 1494 2483
510 1494 2483
520 1494 2483
530 1494 2483
540 1494 2483
550 1494 2483
560 1494 2483
570 1494 2483
580 1494 2483
590 1494 2485
600 1494 2486
610 1494 2486
620 1494 2486
630 1494 2486
640 1494 2486
650 1494 2486
660 1494 2486
670 1494 2486
680 1494 2486
690 1494 2486
700 1494 2486
710 1494 2486
720 14

In [440]:
dists3.sort()
min_dist_diff = None
for dist3_idx in range(1, len(dists3)):
    cur_dist_diff = dists3[dist3_idx] - dists3[dist3_idx - 1]
    if min_dist_diff is None:
        min_dist_diff = 1
    min_dist_diff = min(min_dist_diff, cur_dist_diff)
print(min_dist_diff)

1.0416250129097193e-07


In [ ]:
# for t in trinities:
#     assert (sum([colors[v] for v in t]) % 5 == 0)
# for v in opposite_points:
#     if v < opposite_points[v]:
#         assert (sum([colors[v], colors[opposite_points[v]]]) % 5 == 0)

In [ ]:
# for t in trinities2:
#     assert (sum([colors[v] for v in t]) % 6 == 0)
# for v in opposite_points2:
#     if v < opposite_points2[v]:
#         assert (sum([colors[v], colors[opposite_points2[v]]]) % 6 == 0)